In [1]:
from FinMind.data import DataLoader 
import pandas as pd



In [45]:
## Get data
stock_no = '006208'
dl = DataLoader()
stock_data = dl.taiwan_stock_daily(
    stock_id=stock_no, 
    start_date='2020-06-01',
    end_date='2024-09-30')
stock_data.to_csv('stock_data.csv', index=False)
print(stock_data.keys())


2024-09-27 14:47:14.051 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPrice, data_id: 006208


Index(['date', 'stock_id', 'Trading_Volume', 'Trading_money', 'open', 'max',
       'min', 'close', 'spread', 'Trading_turnover'],
      dtype='object')


In [47]:
## calculate MA
# df = pd.read_csv('stock_data.csv')
df = stock_data

def calculate_ma(data, window):
    return data.rolling(window=window).mean()
    
ma_window = 60
df[f'MA_{ma_window}'] = df['close'].rolling(window=ma_window).mean()
df.to_csv('stock_data.csv', index=False)

print(df[[f'MA_{ma_window}', 'close']].tail())




           MA_60   close
1024  106.654167  107.00
1025  106.650833  108.05
1026  106.660833  109.80
1027  106.680000  110.55
1028  106.708333  110.30


In [48]:
## 找出 均線黃金交叉 與 均線死亡交叉, 以 50 與 200 為例
df['MA_200'] = df['close'].rolling(window=200).mean()
df['MA_60'] = df['close'].rolling(window=60).mean()

df['is_golden_cross'] = (df['MA_60'].shift(1) < df['MA_200'].shift(1)) & (df['MA_60'] > df['MA_200'])
df['is_death_cross'] = (df['MA_60'].shift(1) > df['MA_200'].shift(1)) & (df['MA_60'] < df['MA_200'])

print(f'黃金交叉:\n{df[df['is_golden_cross']]['date']} \n')
print(f'死亡交叉:\n{df[df['is_death_cross']]['date']} \n')

黃金交叉:
650    2023-03-07
Name: date, dtype: object 

死亡交叉:
451    2022-04-27
Name: date, dtype: object 



In [49]:
## 買入信號：前一天的關盤價 < 前一天的 MA 且 當天的關盤價 > 當天的 MA
df['is_buy_signal'] = (df['close'].shift(1) < df[f'MA_{ma_window}'].shift(1)) & (df['close'] > df[f'MA_{ma_window}']) 


In [50]:
## 賣出信號: 前一天的關盤價 > 前一天的 MA 且 當天的關盤價 < 當天的 MA
df['is_sell_signal'] = (df['close'].shift(1) > df[f'MA_{ma_window}'].shift(1)) & (df['close'] < df[f'MA_{ma_window}']) 


In [44]:
# 找出所有買入信號的日期
buy_dates = df[df['is_buy_signal']][['date', f'MA_{ma_window}', 'close']]

# 找出所有賣出信號的日期
sell_dates = df[df['is_sell_signal']][['date', f'MA_{ma_window}', 'close']]

print("買入信號日期：")
print(f'{buy_dates.tail()}')

print("\n賣出信號日期：")
print(f'{sell_dates.tail()}')

買入信號日期：
            date       MA_60   close
792   2023-10-11   72.821667   72.85
809   2023-11-03   72.043333   72.40
999   2024-08-16  106.035000  107.35
1007  2024-08-28  106.928333  106.95
1024  2024-09-23  106.654167  107.00

賣出信號日期：
            date       MA_60   close
749   2023-08-08   73.573333   73.40
797   2023-10-18   72.613333   72.10
989   2024-08-02  105.058333  102.40
1002  2024-08-21  106.395000  106.25
1008  2024-08-29  107.038333  105.75


In [53]:
# EMA
df = pd.read_csv('stock_data.csv')
ema_window = 60 
df[f'EMA_{ema_window}'] = df['close'].ewm(span=ema_window).mean()

print(df[[f'EMA_{ema_window}', 'close']].tail())

          EMA_60   close
1024  104.326714  107.00
1025  104.448789  108.05
1026  104.624238  109.80
1027  104.818526  110.55
1028  104.998246  110.30


In [54]:
df['is_buy_signal_ema'] = (df['close'].shift(1) < df[f'EMA_{ema_window}'].shift(1)) & (df['close'] > df[f'EMA_{ema_window}'])

df['is_sell_signal_ema'] = (df['close'].shift(1) > df[f'EMA_{ema_window}'].shift(1)) & (df['close'] < df[f'EMA_{ema_window}'])

In [55]:
print(f'買入信號日期：\n{df[df["is_buy_signal_ema"]][["date", f"EMA_{ema_window}", "close"]].tail()}')

print(f'賣出信號日期：\n{df[df["is_sell_signal_ema"]][["date", f"EMA_{ema_window}", "close"]].tail()}')

買入信號日期：
            date      EMA_60   close
809   2023-11-03   72.049896   72.40
862   2024-01-18   74.870864   75.00
995   2024-08-12  104.007069  104.05
1019  2024-09-13  104.099834  104.40
1022  2024-09-19  104.155962  105.35
賣出信號日期：
            date      EMA_60  close
797   2023-10-18   72.414961   72.1
861   2024-01-17   74.866487   74.7
989   2024-08-02  104.883877  102.4
1012  2024-09-04  104.683908  100.3
1021  2024-09-18  104.115486  103.9
